In [ ]:
import torch
import jax.numpy as jnp
from jax import grad
from jax import jit
import trio

# Byte Pair Encoding Tokenization

Based off of Andrej Kapathy YouTube Video "Let's Build GPT Tokenizer"

``` Place holder for BPE Description```

In [3]:
def find_pairs(input):
    encoded_input = input.encode("utf-8")
    encoded_input = list(map(int, encoded_input))

    counts = {}

    for pair in zip(encoded_input, encoded_input[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [ ]:
pairs = find_pairs("This is a test string. It has no meaning and serves for testing purposes only. I'll also add the common sentence with every letter next. The quick brown fox jumps over the lazy dog.")

most_freq_pair = max(pairs, key = pairs.get)

def merge_most_frequent(id_list, pair, new_token):
    tokens = []
    i = 0
    while i < len(id_list):
        if i < len(id_list) - 1 and id_list[i] == pair[0] and id_list[i+1] == pair[1]:
            tokens.append(new_token)
            i += 2
        else:
            tokens.append(id_list[i])
            i += 1
    return tokens

# test
print(merge_most_frequent([5, 6, 6, 7, 9, 1], (6, 7), 99))

[5, 6, 99, 9, 1]


In [ ]:
vocab_size = 276